### 1. Train

In [3]:
TRAIN_FOLDER = './data/train'
VAL_FOLDER = './data/val'
TEST_FOLDER = './data/test'
IMAGE_SIZE = 224

ALPHA = 0.25
EPOCHS = 5000
PATIENCE = 100

In [4]:
import csv
import math

import cv2
from keras.applications.mobilenet import MobileNet, _depthwise_conv_block
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import *
from keras.models import *
from keras.preprocessing.image import *
from keras.utils import Sequence


class DataSequence(Sequence):

    def __load_images(self, dataset):
        out = []
        for file_name in dataset:
            im = cv2.resize(cv2.imread(file_name), (self.image_size, self.image_size))
            out.append(im)

        return np.array(out)

    def __init__(self, csv_file, image_path, image_size, batch_size=32, feature_scaling=False):
        self.csv_file = csv_file
        with open(self.csv_file, "r") as file:
            reader = csv.reader(file, delimiter=",")
            arr = list(reader)

        self.y = np.zeros((len(arr), 8))
        self.x = []
        self.image_size = image_size

        # for index, (path, class_id, width, height, x0, y0, x1, y1) in enumerate(arr):
        for index, coner_points in enumerate(arr):
            self.y[index] = np.array(coner_points)
            self.x.append(f'{image_path}/object_image{index}.png')

        self.batch_size = batch_size
        self.feature_scaling = feature_scaling
        if self.feature_scaling:
            dataset = self.__load_images(self.x)
            broadcast_shape = [1, 1, 1]
            broadcast_shape[2] = dataset.shape[3]

            self.mean = np.mean(dataset, axis=(0, 1, 2))
            self.mean = np.reshape(self.mean, broadcast_shape)
            self.std = np.std(dataset, axis=(0, 1, 2))
            self.std = np.reshape(self.std, broadcast_shape) + K.epsilon()

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        images = self.__load_images(batch_x).astype('float32')
        if self.feature_scaling:
            images -= self.mean
            images /= self.std

        return images, batch_y
    
def create_mobile_net_model(size, alpha):
    mobile_net = MobileNet(input_shape=(size, size, 3), include_top=False, alpha=alpha)
    x = _depthwise_conv_block(mobile_net.layers[-1].output, 1024, alpha, 1, block_id=14)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(4, 4))(x)
    x = Conv2D(8, kernel_size=(1, 1), padding="same")(x)
    x = Reshape((8,))(x)
    return Model(inputs=mobile_net.input, outputs=x)


def train(model, epochs, image_size):
    train_datagen = DataSequence("./data/train/corners.csv", TRAIN_FOLDER, image_size)
    validation_datagen = DataSequence("./data/val/corners.csv", VAL_FOLDER, image_size)

    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
    checkpoint = ModelCheckpoint("model-{val_acc:.2f}.h5", monitor="val_acc", verbose=1, save_best_only=True,
                                 save_weights_only=True, mode="auto", period=1)
    stop = EarlyStopping(monitor="val_acc", patience=PATIENCE, mode="auto")

    model.fit_generator(train_datagen, steps_per_epoch=1028, epochs=epochs, validation_data=validation_datagen,
                        validation_steps=22, callbacks=[checkpoint, stop])

model = create_mobile_net_model(IMAGE_SIZE, ALPHA)
train(model, EPOCHS, IMAGE_SIZE)

Epoch 1/5000
1028/1028 [==============================] - 619s 602ms/step - loss: 232.8792 - acc: 0.9722 - val_loss: 5485.4912 - val_acc: 0.1000

Epoch 00001: val_acc improved from -inf to 0.10000, saving model to model-0.10.h5
Epoch 2/5000
 179/1028 [====>.........................] - ETA: 8:15 - loss: 0.0232 - acc: 1.0000

KeyboardInterrupt: 

### 2. eval

In [8]:
import shutil
import glob
import pathlib

OUTPUT_TEST_PATH = './data/output_test'
if os.path.isdir(OUTPUT_TEST_PATH):
    shutil.rmtree(OUTPUT_TEST_PATH)
model = create_mobile_net_model(IMAGE_SIZE, ALPHA)
model.load_weights('model-0.10.h5')

image_paths = sorted(glob.glob('{}/*png'.format(VAL_FOLDER)))
for i in range(len(image_paths)):
    image_path = image_paths[i]
    image = cv2.resize(cv2.imread(image_path), (IMAGE_SIZE, IMAGE_SIZE))
    points = model.predict(x=np.array([image]))[0].astype(int)
    points = np.array([[points[0], points[1]], 
                       [points[2], points[3]], 
                       [points[6], points[7]], 
                       [points[4], points[5]]])
#     print('points:', points)
    
#     with open('data/train/corners.csv', "r") as file:
#         reader = csv.reader(file, delimiter=",")
#         arr = list(reader)
#         points = [np.int32(float(str)) for str in arr[0]]
#         points = np.array([[points[0], points[1]], 
#                            [points[2], points[3]], 
#                            [points[6], points[7]], 
#                            [points[4], points[5]]])
#         print('points:', points)
    
    cv2.polylines(image, [points], True, (255,255,255))
    pathlib.Path(OUTPUT_TEST_PATH).mkdir(parents=True, exist_ok=True)
    cv2.imwrite(f'{OUTPUT_TEST_PATH}/image{i}.png', image)